In [ ]:
import sqlite3
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

# Database setup
def initialize_db():
    conn = sqlite3.connect("products.db")
    cursor = conn.cursor()
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            price REAL NOT NULL
        )
        """
    )
    conn.commit()
    conn.close()

def add_product(name, price):
    conn = sqlite3.connect("products.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO products (name, price) VALUES (?, ?)", (name, price))
    conn.commit()
    conn.close()

def search_products(keyword):
    conn = sqlite3.connect("products.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM products WHERE name LIKE ?", (f"%{keyword}%",))
    results = cursor.fetchall()
    conn.close()
    return results

def update_product(product_id, name, price):
    conn = sqlite3.connect("products.db")
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE products SET name = ?, price = ? WHERE id = ?",
        (name, price, product_id),
    )
    conn.commit()
    conn.close()

def delete_product(product_id):
    conn = sqlite3.connect("products.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM products WHERE id = ?", (product_id,))
    conn.commit()
    conn.close()

def fetch_all_products():
    conn = sqlite3.connect("products.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM products")
    results = cursor.fetchall()
    conn.close()
    return results

# GUI setup
class ProductApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Product Management")

        # UI Elements
        self.name_label = tk.Label(root, text="Product Name:")
        self.name_label.grid(row=0, column=0, padx=10, pady=5)
        self.name_entry = tk.Entry(root)
        self.name_entry.grid(row=0, column=1, padx=10, pady=5)

        self.price_label = tk.Label(root, text="Price:")
        self.price_label.grid(row=1, column=0, padx=10, pady=5)
        self.price_entry = tk.Entry(root)
        self.price_entry.grid(row=1, column=1, padx=10, pady=5)

        self.add_button = tk.Button(root, text="Add", command=self.add_product)
        self.add_button.grid(row=0, column=2, padx=10, pady=5)

        self.update_button = tk.Button(root, text="Update", command=self.update_product)
        self.update_button.grid(row=1, column=2, padx=10, pady=5)

        self.delete_button = tk.Button(root, text="Delete", command=self.delete_product)
        self.delete_button.grid(row=2, column=2, padx=10, pady=5)

        self.search_label = tk.Label(root, text="Search:")
        self.search_label.grid(row=2, column=0, padx=10, pady=5)
        self.search_entry = tk.Entry(root)
        self.search_entry.grid(row=2, column=1, padx=10, pady=5)

        self.search_button = tk.Button(root, text="Search", command=self.search_products)
        self.search_button.grid(row=2, column=3, padx=10, pady=5)

        self.product_list = ttk.Treeview(root, columns=("ID", "Name", "Price"), show="headings")
        self.product_list.heading("ID", text="ID")
        self.product_list.heading("Name", text="Name")
        self.product_list.heading("Price", text="Price")
        self.product_list.grid(row=3, column=0, columnspan=4, padx=10, pady=10)

        self.load_products()

    def add_product(self):
        name = self.name_entry.get()
        price = self.price_entry.get()
        if name and price:
            try:
                add_product(name, float(price))
                self.load_products()
                messagebox.showinfo("Success", "Product added successfully!")
            except ValueError:
                messagebox.showerror("Error", "Invalid price value.")
        else:
            messagebox.showerror("Error", "Please fill in all fields.")

    def search_products(self):
        keyword = self.search_entry.get()
        results = search_products(keyword)
        self.update_product_list(results)

    def update_product(self):
        selected_item = self.product_list.selection()
        if selected_item:
            product_id = self.product_list.item(selected_item[0])['values'][0]
            name = self.name_entry.get()
            price = self.price_entry.get()
            if name and price:
                try:
                    update_product(product_id, name, float(price))
                    self.load_products()
                    messagebox.showinfo("Success", "Product updated successfully!")
                except ValueError:
                    messagebox.showerror("Error", "Invalid price value.")
            else:
                messagebox.showerror("Error", "Please fill in all fields.")
        else:
            messagebox.showerror("Error", "Please select a product to update.")

    def delete_product(self):
        selected_item = self.product_list.selection()
        if selected_item:
            product_id = self.product_list.item(selected_item[0])['values'][0]
            delete_product(product_id)
            self.load_products()
            messagebox.showinfo("Success", "Product deleted successfully!")
        else:
            messagebox.showerror("Error", "Please select a product to delete.")

    def load_products(self):
        products = fetch_all_products()
        self.update_product_list(products)

    def update_product_list(self, products):
        for row in self.product_list.get_children():
            self.product_list.delete(row)
        for product in products:
            self.product_list.insert("", "end", values=product)

if __name__ == "__main__":
    initialize_db()
    root = tk.Tk()
    app = ProductApp(root)
    root.mainloop()
